In [ ]:
from astropy.io import fits
from io import BytesIO
import sys
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt
sys.path.append('/home/bekah/gPhoton2')
from gPhoton.coadd import zero_flag_and_edge
import pyarrow.parquet as parquet


In [ ]:
def centile_clip(image, centiles=(0, 90)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    import numpy as np
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)

    if isinstance(image, np.ma.MaskedArray):
        return result

    return result.data

In [ ]:
photons = parquet.read_table("/home/bekah/Downloads/e00848-fd-b00.parquet").to_pandas()
#photons = parquet.read_table("/home/bekah/gPhoton2/test_data/e23456/e23456-nd-b00.parquet").to_pandas()

In [ ]:
photons

In [ ]:
time = abs(photons['t'].iloc[0]-photons['t'].iloc[-1])

In [ ]:
photons = photons[(photons['detrad'] < 600)]
photons = photons.dropna(subset=['ra', 'dec'])
photons = photons[(photons['flags'] < 1)]

In [ ]:
photons['q'].describe()

In [ ]:
from quickbin import bin2d

ra = bin2d(photons['col'],photons['row'],photons['ra'],n_bins=800,op=('count', 'std'))
dec = bin2d(photons['col'],photons['row'],photons['dec'],n_bins=800,op=('std'))
q = bin2d(photons['col'],photons['row'],photons['q'],n_bins=800,op=('std', 'mean'))
q_m =  bin2d(photons['col'],photons['row'],photons['q'],n_bins=800,op=('min', 'max'))
q_med = bin2d(photons['col'],photons['row'],photons['q'],n_bins=800,op=('median'))

In [ ]:
disp = ra['std']+dec

In [ ]:
plt.scatter(q_m['max'],ra['count']/time,s=.1)

In [ ]:
plt.scatter(q_m['max'],disp,s=1,c=ra['count'])

In [ ]:
plt.imshow(q_m['max']-q_m['min'])

In [ ]:
plt.scatter(disp,ra['count']/time,s=.1)
plt.ylabel("total photons per bin div by expt")
plt.xlabel('dispersion per bin')

In [ ]:
plt.scatter(q['mean'],ra['count']/time,s=.1,marker='.')
plt.ylabel("total photons per bin div by expt")
plt.xlabel('mean pulse height per bin')


In [ ]:
plt.scatter(q_m['max']-q_m['min'],ra['count']/time,s=1,marker='.')
plt.ylabel("total photons per bin div by expt")
plt.xlabel('mean pulse height per bin')

In [ ]:
plt.scatter(q_m['max'],ra['count']/time,s=.01,marker='.')
plt.ylabel("total photons per bin div by expt")
plt.xlabel('median pulse height per bin')
plt.ylim(0,.1)

In [ ]:
plt.hist(q['mean'].ravel(),bins=100)

In [ ]:
plt.scatter(q_med,ra['count']/time,s=.01)
plt.ylabel("total photons per bin div by expt")
plt.xlabel('max pulse height per bin')
plt.ylim(0,.1)

In [ ]:
plt.hist(ra['std'].ravel())

In [ ]:
plt.hist(ra['count'].ravel(),bins=100)

In [ ]:
plt.scatter(q_m['max']-q_m['min'],q['mean'],s=.1,c=ra['count'])

In [ ]:
plt.imshow(q_m['min'])

In [ ]:
plt.imshow(q_m['max']-q_m['min'])

In [ ]:
plt.imshow(q_med)

In [ ]:
plt.imshow(disp)

In [ ]:
plt.imshow(centile_clip(ra['count']))
plt.colorbar()

In [ ]:
disp

In [ ]:
photons.keys()